In [1]:
import torch
import torch.nn as nn
from torchquad import VEGAS 
import numpy as np 

18:14:20|TQ-INFO| Initializing torchquad.


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(1,10),
        nn.Sigmoid(),
        nn.Linear(10,1, bias=False)
        )
        
    def forward(self, x):
        N = self.layers(x)
        return N

In [3]:
model = NeuralNetwork()
upper_r = 6
lower_r = 1e-2
steps = 100
R_train = torch.Tensor(np.linspace(lower_r, upper_r, steps)[:,None])
beta = 2
global Z
Z = 1
global e
#e = -1.602e-19
e = -1
global hbar
#hbar = 1.054e-34
hbar = 1
global m
#m = 9.109e-31
m = 1
global l
l = 0
V = lambda r: -(Z*e**2)/r
Phi_t = lambda r: torch.exp(-beta*r**2) * model.forward(r)

In [4]:
vg = VEGAS()
integral_value = vg.integrate(Phi_t, dim=1, N=200, integration_domain=[[-6,6]])
integral_value

/home/andresgo/anaconda3/lib/python3.9/site-packages/torchquad/integration/vegas_stratification.py:113: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = tmp // self.N_strat
18:14:20|TQ-INFO| Iteration 1, Acc=nan, Result=-3.6274e-01,neval=30
18:14:20|TQ-INFO| Iteration 2, Acc=nan, Result=-1.7175e-01,neval=64
18:14:20|TQ-INFO| Iteration 3, Acc=nan, Result=-2.5581e-01,neval=98
18:14:20|TQ-INFO| Iteration 4, Acc=nan, Result=-4.3818e-01,neval=132
18:14:20|TQ-INFO| Iteration 5, Acc=nan, Result=-4.9716e-01,neval=162
18:14:20|TQ-INFO| Iteration 6, Acc=nan, Result=-3.2262e-01,neval=216
18:14:20|TQ-INFO| Computed integral after 216 evals was -3.22624654e-01.


tensor(-0.3226, grad_fn=<DivBackward0>)

In [5]:
f = lambda x: -x**2
integral_value1 = vg.integrate(f, dim=1, N=100000, integration_domain=[[0,1]])
integral_value1

18:14:20|TQ-INFO| Iteration 1, Acc=nan, Result=-3.3333e-01,neval=15000
18:14:20|TQ-INFO| Iteration 2, Acc=nan, Result=-3.3334e-01,neval=25910
18:14:21|TQ-INFO| Iteration 3, Acc=nan, Result=-3.3332e-01,neval=35248
18:14:21|TQ-INFO| Iteration 4, Acc=nan, Result=-3.3334e-01,neval=44652
18:14:21|TQ-INFO| Iteration 5, Acc=nan, Result=-3.3334e-01,neval=54022
18:14:21|TQ-INFO| Iteration 6, Acc=nan, Result=-3.3330e-01,neval=73100
18:14:21|TQ-INFO| Iteration 7, Acc=nan, Result=-3.3332e-01,neval=92240
18:14:21|TQ-INFO| Computed integral after 92240 evals was -3.33310425e-01.


tensor(-0.3333)

In [6]:
integral_value2 = vg.integrate(lambda r: Phi_t(r)**2, dim=1, N=100000, integration_domain=[[0,1]])
integral_value2

18:14:21|TQ-INFO| Iteration 1, Acc=1.7273e-06, Result=3.6978e-02,neval=15000
18:14:21|TQ-INFO| Iteration 2, Acc=1.2380e-06, Result=3.6978e-02,neval=26184
18:14:21|TQ-INFO| Iteration 3, Acc=2.7876e-06, Result=3.6979e-02,neval=35404
18:14:22|TQ-INFO| Iteration 4, Acc=3.0977e-06, Result=3.6980e-02,neval=44854
18:14:22|TQ-INFO| Iteration 5, Acc=3.9321e-06, Result=3.6978e-02,neval=54264
18:14:22|TQ-INFO| Iteration 6, Acc=4.2156e-06, Result=3.6978e-02,neval=73256
18:14:22|TQ-INFO| Iteration 7, Acc=4.7325e-06, Result=3.6980e-02,neval=92308
18:14:22|TQ-INFO| Computed integral after 92308 evals was 3.69788334e-02.


tensor(0.0370, grad_fn=<DivBackward0>)

In [7]:
import vegas
import math

def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.85(49)        0.85(49)            0.00     1.00
  2   0.793(75)       0.794(74)           0.01     0.91
  3   1.07(18)        0.833(69)           0.97     0.38
  4   1.100(75)       0.955(51)           2.95     0.03
  5   1.016(33)       0.998(28)           2.46     0.04
  6   0.999(24)       0.998(18)           1.97     0.08
  7   0.985(19)       0.992(13)           1.68     0.12
  8   1.024(15)       1.0061(98)          1.82     0.08
  9   0.989(14)       1.0005(80)          1.71     0.09
 10   0.990(12)       0.9970(66)          1.58     0.11

result = 0.9970(66)    Q = 0.11


In [8]:
import torch.nn as nn
import torch
import numpy as np
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(1,10),
        nn.Sigmoid(),
        nn.Linear(10,1, bias=False)
        )
        
    def forward(self, x):
        N = self.layers(x)
        return N

In [9]:
model = NeuralNetwork()

V = lambda r: -(Z*e**2)/r

def Phi_t(r):
    r = torch.Tensor(r)
    return torch.exp(-2*r**2) * model.forward(r).detach().numpy()

In [10]:
integ = vegas.Integrator([[-6,6]])
result = integ(Phi_t, nitn=10, neval=1000)
result

RAvgArray([0.172132(45)], dtype=object)

In [11]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Linear)):
            nn.init.kaiming_normal_(m.weight.data)

model = NeuralNetwork()
initialize_weights(model)
result = integ(Phi_t, nitn=10, neval=1000)
result

RAvgArray([0.83618(33)], dtype=object)